In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
import os
import time 

# Step 1: Capture Images for Multiple People

def capture_images_for_person(name, num_images=20, img_size=(64, 64), delay=0.5):

    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    cap = cv2.VideoCapture(0)
    
    os.makedirs(f"dataset/{name}", exist_ok=True)
    print(f"Capturing images for {name}...")
    
    count = 0
    while count < num_images:
        ret, frame = cap.read()
        if not ret:
            print("Failed to capture video frame.")
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

        for (x, y, w, h) in faces:
            # Extract the face ROI and resize it
            face_roi = frame[y:y+h, x:x+w]
            face_resized = cv2.resize(face_roi, img_size)
            
            # Save the face image
            image_path = f"dataset/{name}/{name}_{count}.jpg"
            cv2.imwrite(image_path, face_resized)
            count += 1
            cv2.putText(frame, f"Capturing {count}/{num_images} images", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

            break

        cv2.imshow(f"Capture Images for {name}", frame)

        # Add a delay between captures
        time.sleep(delay)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    print(f"Captured {count} images for {name}.")
    cap.release()
    cv2.destroyAllWindows()


def build_cnn(input_shape, num_classes):
    
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(num_classes, activation='softmax')  # Output layer with 'num_classes' neurons
    ])
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def load_data(dataset_dir, img_size=(64, 64)):
    
    images = []
    labels = []
    label_mapping = {}
    current_label = 0

    for person_name in os.listdir(dataset_dir):
        person_dir = os.path.join(dataset_dir, person_name)
        if os.path.isdir(person_dir):
            label_mapping[current_label] = person_name
            for img_name in os.listdir(person_dir):
                img_path = os.path.join(person_dir, img_name)
                image = cv2.imread(img_path)
                if image is not None:
                    image_resized = cv2.resize(image, img_size) / 255.0
                    images.append(image_resized)
                    labels.append(current_label)
            current_label += 1

    images = np.array(images)
    labels = np.array(labels)

    # One-hot encode labels
    labels = np.eye(len(label_mapping))[labels]

    return images, labels, label_mapping

def train_model(dataset_dir, img_size=(64, 64)):
    
    images, labels, label_mapping = load_data(dataset_dir, img_size)
    num_classes = len(label_mapping)

    model = build_cnn(input_shape=(img_size[0], img_size[1], 3), num_classes=num_classes)
    model.fit(images, labels, epochs=10, batch_size=8)

    model.save("face_recognition_model.h5")
    np.save("label_mapping.npy", label_mapping)

    print("Model training complete. Saved model and label mapping.")
    return model, label_mapping

# Step 3: Real-Time Face Recognition

def recognize_faces(video_source=0, img_size=(64, 64)):

    # Load the trained model and label mapping
    from tensorflow.keras.models import load_model
    model = load_model("face_recognition_model.h5")
    label_mapping = np.load("label_mapping.npy", allow_pickle=True).item()

    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    cap = cv2.VideoCapture(video_source)

    print("Starting video stream... Press 'q' to quit.")
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to capture video frame.")
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

        for (x, y, w, h) in faces:
            face_roi = frame[y:y+h, x:x+w]
            face_resized = cv2.resize(face_roi, img_size) / 255.0
            face_resized = face_resized.reshape(1, img_size[0], img_size[1], 3)


            predictions = model.predict(face_resized)
            predicted_label = np.argmax(predictions)
            confidence = predictions[0][predicted_label]

            if confidence > 0.6:  # Threshold for recognition
                name = label_mapping[predicted_label]
            else:
                name = "Unknown"

            
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.putText(frame, f"{name} ({confidence:.2f})", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        cv2.imshow("Face Recognition", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [ ]:
capture_images_for_person("Reem", num_images=20)

In [ ]:
# capture_images_for_person("name", num_images=30)

In [ ]:
model, label_mapping = train_model("dataset")

In [ ]:
recognize_faces()